# CSE 5243 - Introduction to Data Mining
## Homework 2: Classification
- Instructor: Davis Buenger
- Section: Tuesday/Thursday 5:30 pm
- Student Name: Xidan Kou
- Student Email: kou.55@osu.edu
- Student ID: 500339314

Template Version V1.
***

**Instructions and Helpful Hints:**
- Consider putting all of your "discussion" text in markdown cells, not inline with code. That gives you more control over formatting. Markdown cheat sheet: https://www.markdownguide.org/cheat-sheet
- Explain what you are doing, and why.  Explain what you found out or learned.
- *Make sure you run your entire workbook before handing it in, so the output cells are populated.*
- Follow the Section structure as much as possible - put your content where it is requested, so we can find your answers.
- If you have questions on expectations or need clarification or guidance, please ask.  Post to Piazza if it is a general question, so everyone benefits.

***
# Section: Overview
- Insert a short description of the scope of this exercise, any supporting information, etc.
***

This exercise is to come up with the effective model based on the cost of misclassification to maximize the profit of the Pampered Pets Inc.. The model will help company to determine the potential customer and further promote the appropiate products. To be specific, this exercise will first classify people into categories of whether they have dog or cat or both based on several different attributes. The model will be build by three different methods and further determine which one works the best for this specific situation.

***
# Section: Setup
- Add any needed imports, helper functions, etc., here.
***

In [117]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from matplotlib import cm
import seaborn as sns

***
# Section: 1 - Evaluation mMethod
- Define measures for evaluating the classification models you develop.  Explain why the measures you choose provide a useful view into the value and usefulness of the model you eventually chose for the company to use.  Define two types:
***

***
## Section: 1.1 - Define measures that do not include the cost information
- (e.g., confusion matrices, accuracy, precision, recall, F-measures, etc.).
***

Without considering the cost information, accurary will be important, because we want the model with overall the highest accurary. Also, F-measures would be a good statistics to look at the overall model performance, since it is the ratio of precision and recall. We would like the F-measures to be as close to 1.

***
## Section: 1.2 - Define measures that do include the cost information
- (e.g., using cost matrices).
***

Considering the cost matrices, since Prospects who are mis-classified (e.g., a dog owner classified as a cat owner) will not
buy anything and will hang up immediately, we want FPs and FNs to be as least as possible in confusion matrix.

***
# Section: 2 - Pre-Processing of the Dataset
- Use the Adult dataset.  Split it into a Training dataset and a Test dataset based on the Train attribute (see the README worksheet in the data file).  Keep them separate and use the Training dataset for training/tuning and the Test dataset for testing.
***

***
## Section: 2.1 - Explore the attributes
- As in Homework 1, explore the attributes briefly. Reference the provided Data Dictionary (CHIS 2003 PUF Data Dictionary-Adult May2010.pdf).
- Provide basic statistics for the attributes.
- List which attributes are Nominal (even though they are encoded as numbers), Ordinal, Interval, Ratio.
***

Loading data:

In [40]:
adult = pd.read_excel("adult.xlsx")
adult.head(10)

,RecID,Train,Class,Health,Spouse,Education,Income,Home Type,Number of Rooms,Own or Rent,...,Race,Occupation,Age,Gender,House Tenure,Months at Current Address,Weight,Walking,Employment Status,Years in USA
0,1,y,c,1,1,4,50000,1,6,1,...,5,-1,35,2,1,48,220,1,5,-1
1,2,y,d,2,-1,4,13200,3,3,2,...,5,-1,81,2,2,60,140,1,5,-1
2,3,y,n,2,1,3,12000,1,3,2,...,2,-1,22,2,2,36,120,2,5,-1
3,4,y,c,1,-1,1,8000,1,6,2,...,2,-1,62,1,2,144,175,2,5,-1
4,5,y,d,1,1,7,65000,1,5,1,...,7,-1,41,1,1,144,180,1,5,-1
5,6,y,b,1,1,4,70000,1,6,1,...,5,5,45,2,1,204,200,1,1,-1
6,7,y,n,5,-1,4,12000,4,2,1,...,5,-1,85,2,1,120,98,2,5,-1
7,8,y,c,3,1,3,13000,1,5,2,...,5,-1,37,2,2,36,145,2,5,-1
8,9,y,c,1,-1,6,9000,1,6,2,...,5,3,37,2,2,1,200,2,1,-1
9,10,y,d,1,1,3,75000,1,6,1,...,5,5,48,2,1,228,130,1,1,-1


Statistics of data:

In [ ]:
adult.describe()

Attributes type and description:

- RecID: Interval, artificially created variable to keep track the row
- Train: Nominal, artificially created variable to keep track of the training and testing dataset
- Class: Nominal, artificially created variable to keep track of each categories, where n = Neither Cats nor Dogs are allowed inside home, c = Cats are allowed inside home, but Dogs are not,d = Dogs are allowed inside home, but Cats are not, b = Both Cats and Dogs are allowed inside home
- Health: Ordinal, GENERAL HEALTH CONDITION, where '1' = EXCELLENT, '5' = POOR
- Spouse: Nomial, SPOUSE LIVING IN SAME HOUSEHOLD, where '-1' = INAPPLICABLE, '1' = YES, '2' = NO
- Education: Nominal, EDUCATIONAL ATTAINMENT, where '1'= GRADE 1-8,'2' = GRADE 9-11,3 = GRADE 12 / H.S. DIPLOMA,4 = SOME COLLEGE,5 = VOCATIONAL SCHOOL,6 = AA OR AS DEGREE,7 = BA OR BS DEGREE,8 = SOME GRAD. SCHOOL,9 = MA OR MS DEGREE,10 = PH.D. OR EQUIVALENT,91 = NO FORMAL EDUCATION
- Income: Interval, HOUSEHOLD TOTAL ANNUAL INC (PUF RECODE)
- Home type: Nominal, LIVE IN HOUSE, DUPLEX, BUILDING WITH 3+ UNITS, OR MOBILE HOME
- Number of Rooms: Interval, HOUSEHOLD SIZE (PUF RECODE)
- Own or rent: Nominal, OWN OR RENT HOME, where '1' = OWN,'2'= RENT,'3' = OTHER ARRANGEMENT
- Monthly rent: Interval, MONTHLY RENT/SPACE PAYMENT (PUF RECODE)
- Monthly Mortage: MONTHLY MORTGAGE PAYMENT (PUF RECODE) FOR ADULTS WHO OWN A HOME
- Citizenship: Nominal, CITIZENSHIP STATUS (3 LVLS),where '1' = US-BORN CITIZEN,'2' = NATURALIZED CITIZEN, '3'= NON-CITIZEN
- Country of Birth: Nomimal, COUNTRY BORN IN, where'1'= UNITED STATES,'2'= MEXICO,'3' = CENTRAL AMERICA,'4' = OTHER LATIN AMERICA,'5' = ASIA & PACIFIC ISLANDS,'6' = EUROPE,'7' ==OTHER
- Working Status: Nominal, WORKING STATUS, where'-2' =PROXY SKIPPED,'1'= FULL-TIME EMPLOYED (21+ HOURS/WEEK),'2'= PART-TIME EMPLOYED (0-20 HOURS/WEEK),'3'= UNEMPLOYED AND LOOKING FOR WORK,'4'= NOT WORKING
- Family Type: Nominal, FAMILY TYPE (4 LVLS), where '1' =SINGLE NO KIDS,'2' =MARRIED NO KIDS,'3' =MARRIED WITH KIDS,'4' = SINGLE WITH KIDS
- Height :Interval, HEIGHT - INCHES (PUF RECODE)
- Marital Status: Nominal, MARITAL STATUS, where '1' = MARRIED,'2' = OTHERWID/SEP/DIV/LIVING W/PARTNER,'3' = NEVER MARRIED
- Race: Nominal, RACE-CENSUS 2000 DEFINITION
- Occupation: Nominal, MAIN OCCUPATION - 2003 RECODE
- Age: Interval, Age
- Gender: Nominal, '1' = MALE,'2' = FEMALE
- House Tenure: SELF-REPORTED HOUSE TENURE, '1' = RENT, '2' = OWN, '3' = OTHER ARRANGEMENT
- Months at Current Address: Interval, NUMBER OF MONTHS LIVED AT CURRENT ADDRESS
- Weight: Interval, WGHT - POUNDS (PUF RECODE)
- Walking: Nominal, ANY WALKING FOR TRANSPORT OR FUN/EXERCISE
- Employment Status: Nominal, EMPLOYMENT STATUS, '-2' = PROXY SKIPPED,'1' = FULL-TIME EMPLOYED (21+ HOURS/WEEK),'2' = PART-TIME EMPLOYED (0-20 HOURS/WEEK),'3' = EMPLOYED, NOT AT WORK,'4' = UNEMPLOYED AND LOOKING FOR WORK,'5' =  UNEMPLOYED AND NOT LOOKING FOR WORK
- Years in USA: Nominal,YEARS LIVED IN THE US,'-1' =INAPPLICABLE,'1' = <=1 YEAR,'2' = 2-4 YEARS,'3' = 5-9 YEARS,'4' = 10-14 YEARS,'5' = 15+ YEARS

Divide dataset to training dataset and testing dataset

In [79]:
adult_train = adult.loc[adult['Train'] == 'y', ]
adult_test = adult.loc[adult['Train'] == 'n', ]

***
## Section: 2.2 - Revise the dataset
- Review the meanings of the attributes and consider removing redundant or (likely) irrelevant attributes, combining attributes, etc., to reduce the number of attributes.
- (You may choose to use techniques such as those you used in Homework 1 to analyze the impacts of individual attributes on the CLASS attribute, but you need not do a “deep” analysis.)
- Describe what you chose to do (and not do), and why.
***

Finding out highly correlated attributes using correlation matrix

In [72]:
pd.set_option('max_columns', 28)
corr_matrix = adult.corr()
corr_matrix

,RecID,Health,Spouse,Education,Income,Home Type,Number of Rooms,Own or Rent,Monthly Rent,Monthly Mortgage,Citizenship,Country of Birth,Working Status,Family Type,Height,Marital Status,Race,Occupation,Age,Gender,House Tenure,Months at Current Address,Weight,Walking,Employment Status,Years in USA
RecID,1.000000,0.008851,-0.021655,0.009735,-0.001819,0.018983,-0.019032,0.022425,0.038950,-0.007635,0.015306,0.030070,-0.009127,-0.007850,-0.008905,0.024018,-0.059775,0.001179,-0.001001,0.000766,0.024152,0.002240,-0.006227,0.008767,-0.008896,0.012993
Health,0.008851,1.000000,-0.081296,-0.031687,-0.298022,0.125031,-0.178987,0.129316,0.037468,-0.209477,0.141539,0.040790,0.239765,-0.067437,-0.143489,0.036306,-0.003681,-0.049753,0.182648,0.030637,0.130144,0.061830,0.109974,0.039405,0.239786,0.131097
Spouse,-0.021655,-0.081296,1.000000,0.012946,0.274925,-0.230947,0.221762,-0.268454,-0.186748,0.270407,0.088235,0.092068,-0.051239,0.459440,0.052695,-0.891585,-0.017223,0.021119,0.043838,-0.090840,-0.269232,0.032130,0.049142,-0.011189,-0.053151,0.106215
Education,0.009735,-0.031687,0.012946,1.000000,0.078195,-0.017898,0.024369,-0.009649,-0.001868,0.052524,0.059803,0.043993,-0.020326,-0.004017,-0.002370,-0.012872,-0.003162,-0.014457,0.031335,-0.015352,-0.009617,-0.019219,-0.019420,-0.023905,-0.018435,0.056963
Income,-0.001819,-0.298022,0.274925,0.078195,1.000000,-0.260418,0.406292,-0.304064,-0.175815,0.521798,-0.166017,-0.027168,-0.217025,0.119838,0.197455,-0.240227,-0.026587,0.013968,-0.061120,-0.099399,-0.302551,-0.017509,0.039465,-0.020204,-0.220168,-0.120939
Home Type,0.018983,0.125031,-0.230947,-0.017898,-0.260418,1.000000,-0.406330,0.364591,0.408717,-0.335366,0.122205,0.050698,0.042081,-0.126075,-0.061876,0.222853,-0.017049,-0.002344,-0.055083,0.024682,0.361894,-0.243834,-0.022040,-0.019104,0.045433,0.063738
Number of Rooms,-0.019032,-0.178987,0.221762,0.024369,0.406292,-0.406330,1.000000,-0.409101,-0.336354,0.342744,-0.310945,-0.154877,-0.009665,0.074020,0.114557,-0.247186,0.029461,-0.031884,0.203879,0.023271,-0.406438,0.239473,0.048267,0.006343,-0.017090,-0.248939
Own or Rent,0.022425,0.129316,-0.268454,-0.009649,-0.304064,0.364591,-0.409101,1.000000,0.638551,-0.516079,0.190196,0.058886,-0.027665,-0.052365,-0.069893,0.303091,-0.008709,0.029130,-0.274029,0.013282,0.988052,-0.318997,-0.027922,-0.024216,-0.018754,0.104052
Monthly Rent,0.038950,0.037468,-0.186748,-0.001868,-0.175815,0.408717,-0.336354,0.638551,1.000000,-0.436987,0.149778,0.067727,-0.083530,-0.007951,-0.020135,0.212594,-0.021950,0.035251,-0.262970,0.001219,0.632563,-0.321482,-0.033178,-0.022235,-0.075178,0.072581
Monthly Mortgage,-0.007635,-0.209477,0.270407,0.052524,0.521798,-0.335366,0.342744,-0.516079,-0.436987,1.000000,-0.070263,0.019371,-0.212184,0.205762,0.108471,-0.247688,-0.033220,0.031552,-0.126096,-0.046747,-0.510524,-0.104013,0.026602,0.003835,-0.214123,-0.013932


In [87]:
print("highly correlated: ")
for column in corr_matrix.columns: 
    for index in corr_matrix.index: 
        if (column == index):
            break
        elif (corr_matrix[column][index]>0.7):
                print(column, "&", index)
        elif (corr_matrix[column][index]<-0.7):
                print(column, "&", index)

highly correlated: 
Country of Birth & Citizenship
Marital Status & Spouse
House Tenure & Own or Rent
Employment Status & Working Status
Years in USA & Citizenship
Years in USA & Country of Birth


***
## Section: 2.3 - Transform the attributes
- Consider transforming the remaining attributes (e.g., using the data dictionary to replace the numbers with text values for some attributes – this might or might not be useful), normalizing / scaling values, encoding labels (if necessary), etc.
- Describe what you chose to do (and not do), and why.
***

Above are highly correlated attributes, we should delete one of the variable in each combinations. I choose to delete 'Citizenship','Spouse','Working Status','Country of Birth'

In [89]:
adult_sub1 = adult_train.drop(columns = ['Citizenship','Spouse','Working Status','Country of Birth'])
adult_sub1

,RecID,Train,Class,Health,Education,Income,Home Type,Number of Rooms,Own or Rent,Monthly Rent,Monthly Mortgage,Family Type,Height,Marital Status,Race,Occupation,Age,Gender,House Tenure,Months at Current Address,Weight,Walking,Employment Status,Years in USA
0,1,y,c,1,4,50000,1,6,1,-1,600,3,65,1,5,-1,35,2,1,48,220,1,5,-1
1,2,y,d,2,4,13200,3,3,2,170,-1,1,64,2,5,-1,81,2,2,60,140,1,5,-1
2,3,y,n,2,3,12000,1,3,2,350,-1,3,67,1,2,-1,22,2,2,36,120,2,5,-1
3,4,y,c,1,1,8000,1,6,2,300,-1,1,70,2,2,-1,62,1,2,144,175,2,5,-1
4,5,y,d,1,7,65000,1,5,1,-1,1200,3,69,1,7,-1,41,1,1,144,180,1,5,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28024,28025,y,d,2,4,75000,1,4,1,-1,1200,2,65,1,6,2,45,1,1,96,142,1,1,5
28025,28026,y,n,4,5,60000,1,7,1,-1,525,1,70,2,5,-1,59,1,1,384,219,2,5,-1
28026,28027,y,d,3,3,30000,2,5,2,400,-1,1,63,2,5,4,49,2,2,48,160,2,1,-1
28027,28028,y,c,2,2,35000,1,6,1,-1,1145,1,63,2,5,-1,73,2,1,144,128,1,5,5


***
# Section: 3 - Evaluation of the Off-The-Shelf KNN Classifier
- Select the KNN classifier from the SciKit Learn library and run it on the dataset.
***

***
## Section: 3.1 - Configure the off-the-shelf KNN classifier
- Use the KNeighborsClassifier from the SciKit Learn library
- Explain all setup, parameters and execution options you chose to set, and why.
***

Trying 5 nearest neighbors

In [94]:
X_train = adult_train.iloc[:,3:24].values
X_test = adult_test.iloc[:,3:24].values
y_train = adult_train.iloc[:,2].values
y_test = adult_test.iloc[:,2].values

- X_train is the training dataset of 20 dependent variables 
- X_test is the testing dataset of 20 dependent variables
- y_train is the training dataset containing only 1 column and 4 classes
- y_test is the testing dataset containing only 1 column and 4 classes

In [95]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [96]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [97]:
y_pred = classifier.predict(X_test)

In [98]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 250  255  295  499]
 [ 257  370  440 1028]
 [ 355  379  557 1134]
 [ 477  830  978 5911]]
              precision    recall  f1-score   support

           b       0.19      0.19      0.19      1299
           c       0.20      0.18      0.19      2095
           d       0.25      0.23      0.24      2425
           n       0.69      0.72      0.71      8196

    accuracy                           0.51     14015
   macro avg       0.33      0.33      0.33     14015
weighted avg       0.49      0.51      0.50     14015



Trying 20 nearest neighbors

In [126]:
classifier = KNeighborsClassifier(n_neighbors=20)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

In [127]:
y_pred = classifier.predict(X_test)

In [128]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 125  137  266  771]
 [ 122  152  333 1488]
 [ 187  164  459 1615]
 [ 202  276  598 7120]]
              precision    recall  f1-score   support

           b       0.20      0.10      0.13      1299
           c       0.21      0.07      0.11      2095
           d       0.28      0.19      0.22      2425
           n       0.65      0.87      0.74      8196

    accuracy                           0.56     14015
   macro avg       0.33      0.31      0.30     14015
weighted avg       0.48      0.56      0.50     14015



Trying 25 nearest neighbors

In [123]:
classifier = KNeighborsClassifier(n_neighbors=25)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=25)

In [124]:
y_pred = classifier.predict(X_test)

In [125]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[  98  119  263  819]
 [ 117  115  319 1544]
 [ 156  133  442 1694]
 [ 164  225  556 7251]]
              precision    recall  f1-score   support

           b       0.18      0.08      0.11      1299
           c       0.19      0.05      0.09      2095
           d       0.28      0.18      0.22      2425
           n       0.64      0.88      0.74      8196

    accuracy                           0.56     14015
   macro avg       0.32      0.30      0.29     14015
weighted avg       0.47      0.56      0.50     14015



***
## Section: 3.3 - Evaluate the choice of the KNN classifier
- What characteristics of the problem and data made KNN a good or bad choice?
***

For KNN, we would prefer model with k = 20, since it predicts 'b', 'c', 'd' class better, with slightly higher precision rate than other k valus. Overall, since the precision of our usefull information - correctly classify class 'b', 'c', 'd' is low, KNN would not be a good model. The KNN might not be a good choice because our data containing many attributes, and several of them are correlated, attributes may not contribute specificly to certain class. So, it is hard to clearly classify them by only looking at their neighbors.

***
# Section: 4 - Evaluation of Off-The-Shelf Classifier Naive Bayes
- As with the KNN classifier above, choose another classifier from the SciKit Learn library (Decision Tree, SVM, Logistic Regression, etc.) and run it on the dataset.
***

***
## Section: 4.1 - Configure the classifier
- Use the appropriate classifier from the SciKit Learn library.
- Explain all setup, parameters and execution options you chose to set, and why.
***

In [107]:
NB = GaussianNB()
mdl = NB.fit(X_train,y_train)
cls_pred = mdl.predict(X_test) 

array([[ 670,  144,  205,  280],
       [ 756,  291,  316,  732],
       [1015,  197,  536,  677],
       [1480,  626, 1306, 4784]])

In [129]:
confusion_matrix(y_test, cls_pred)
print(classification_report(y_test, cls_pred))

              precision    recall  f1-score   support

           b       0.17      0.52      0.26      1299
           c       0.23      0.14      0.17      2095
           d       0.23      0.22      0.22      2425
           n       0.74      0.58      0.65      8196

    accuracy                           0.45     14015
   macro avg       0.34      0.36      0.33     14015
weighted avg       0.52      0.45      0.47     14015



***
## Section: 4.2 - Run and evaluate the classifier
- Try several values of the parameters (if appropriate) and compare the results.
- Evaluate the performance of the classifier, using the evaluation method you defined above.
***

This model is not desirable since the precision of our usefull information - correctly classify class 'b', 'c', 'd' is low, Naive Bayes would not be a good model. 

***
## Section: 4.3 - Evaluate the choice of the classifier
- What characteristics of the problem and data made the classifier a good or bad choice?
***

The Naive Bayes might not be a good choice because our data containing many attributes, and many of them are correlated, attributes may not contribute specificly to certain class. So, it is hard to clearly classify them by calculation the probobilities.

***
# Section: 5 - Evaluation of Off-The-Shelf SVM
- As with the KNN classifier above, choose another classifier from the SciKit Learn library (Decision Tree, SVM, Logistic Regression, etc.) and run it on the dataset.
***

***
## Section: 5.1 - Configure the classifier
- Use the appropriate classifier from the SciKit Learn library.
- Explain all setup, parameters and execution options you chose to set, and why.
***

In [118]:
model_poly = SVC(kernel ='poly') 
model_poly.fit(X_train, y_train) 

cls_poly_pred = model_poly.predict(X_test)
print(classification_report(y_test,cls_poly_pred))

              precision    recall  f1-score   support

           b       0.18      0.00      0.00      1299
           c       0.25      0.00      0.00      2095
           d       0.30      0.01      0.02      2425
           n       0.59      0.99      0.74      8196

    accuracy                           0.58     14015
   macro avg       0.33      0.25      0.19     14015
weighted avg       0.45      0.58      0.44     14015



***
## Section: 5.2 - Run and evaluate the classifier
- Try several values of the parameters (if appropriate) and compare the results.
- Evaluate the performance of the classifier, using the evaluation method you defined above.
***

The model is slightly desirable than the other two, but still not desirable for this context of problem. 
This model is not desirable since the precision of our usefull information - correctly classify class 'b', 'c', 'd' is low, SVM would not be a good model. 

***
## Section: 5.3 - Evaluate the choice of the classifier
- What characteristics of the problem and data made the classifier a good or bad choice?
***

The SVM might not be a good choice because our data containing many attributes, and many of them are correlated, attributes may not contribute specificly to certain class. So, it is hard to clearly classify them with specific lines.

***
# Section: 6 - Comparison of the Three Classifiers
***

***
## Section: 6.1 - Compare the performance of these classifiers to each other
- What are their strong and weak points?Configure the classifier
***

KKN is useful when the different classes have a clear distance from each other.

***
## Section: 6.2 - Choose a Best Classifier
- Choose one of the three classifiers as best and explain why.
***

I would choose SVM classifier for this specific problem.
Naive Bayes and KNN performs roughly same, with low precision of class 'b', 'c', 'd', around 20%. SVM classifier is slightly better, as it has better precision on class 'd' than the other two model.

***
# Section: 7 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

Different classifier would be suitable for different dataset, we should process the data to determine which one works best.

***
### END-OF-SUBMISSION
***